In [2]:
!pip install pycountry
!pip install pyvis

     ---------------------------------------- 6.2/6.2 MB 56.9 MB/s eta 0:00:00
     ---------------------------------------- 756.0/756.0 kB 24.1 MB/s eta 0:00:00
     ---------------------------------------- 40.7/40.7 kB ? eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [19]:
colaboraciones_2019 = pd.read_csv('colaboraciones_with_names_2019.csv')
colaboraciones_2019 = colaboraciones_2019.drop_duplicates()
colaboraciones_2020 = pd.read_csv('colaboraciones_with_names_2020.csv')
colaboraciones_2020 = colaboraciones_2020.drop_duplicates()
colaboraciones_2021 = pd.read_csv('colaboraciones_with_names_2021.csv')
colaboraciones_2021 = colaboraciones_2021.drop_duplicates()
colaboraciones_2022 = pd.read_csv('colaboraciones_with_names_2022.csv')
colaboraciones_2022 = colaboraciones_2022.drop_duplicates()

authors_country_2019 = pd.read_csv('Authors_with_country_2019.csv')
authors_country_2020 = pd.read_csv('Authors_with_country_2020.csv')

#Mostramos como se estructuran los dataset en general
colaboraciones_2019.head()

,doi,author1,name1,aff1,author2,name2,aff2
0,http://dx.doi.org/10.1109/5gwf.2019.8911693,https://dblp.org/pid/75/4085,Mérouane Debbah,"Khalifa University, Abu Dhabi, United Arab Emi...",https://dblp.org/pid/119/7827,Miguel Ángel Vázquez,Centre Tecnològic de Telecomunicacions de Cata...
1,https://doi.org/10.1109/5gwf.2019.8911693,https://dblp.org/pid/75/4085,Mérouane Debbah,"Khalifa University, Abu Dhabi, United Arab Emi...",https://dblp.org/pid/119/7827,Miguel Ángel Vázquez,Centre Tecnològic de Telecomunicacions de Cata...
2,http://dx.doi.org/10.1007/978-3-030-21074-8_26,https://dblp.org/pid/l/MongLiLee,Mong-Li Lee,National University of Singapore (NUS),https://dblp.org/pid/243/1972-1,Quang Duc Nguyen 0001,"Singapore Eye Research Institute, Singapore"
3,https://doi.org/10.1007/978-3-030-21074-8_26,https://dblp.org/pid/l/MongLiLee,Mong-Li Lee,National University of Singapore (NUS),https://dblp.org/pid/243/1972-1,Quang Duc Nguyen 0001,"Singapore Eye Research Institute, Singapore"
4,http://dx.doi.org/10.1007/978-3-030-21074-8_26,https://dblp.org/pid/l/MongLiLee,Mong-Li Lee,National University of Singapore (NUS),https://dblp.org/pid/h/WynneHsu,Wynne Hsu,National University of Singapore (NUS)


In [20]:
#Agregamos el año a cada dataframe
colaboraciones_2019['year'] = 2019
colaboraciones_2020['year'] = 2020
colaboraciones_2021['year'] = 2021
colaboraciones_2022['year'] = 2022

In [21]:
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

# Add a column with the pairs of authors
pares_autores_2019 = colaboraciones_2019[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2019['pairs'] = pares_autores_2019['name1'] + ' - ' + pares_autores_2019['name2']
pares_autores_2019['pairs2'] = pares_autores_2019['name2'] + ' - ' + pares_autores_2019['name1']



# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2019.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2019.html')


graph_2019.html


In [22]:
# Extrae información única sobre autores y sus afiliaciones
autores_1_2019 = colaboraciones_2019[['name1', 'aff1']].drop_duplicates()
autores_2_2019 = colaboraciones_2019[['name2', 'aff2']].rename(columns={'name2': 'name1', 'aff2': 'aff1'}).drop_duplicates()
autores_2019 = pd.concat([autores_1_2019, autores_2_2019])

# Extraemos el nombre y el páis de cada autor
autores_pais_2019 = authors_country_2019[['Name', 'Country']]

# Crea un diccionario que asigna el país a cada autor
pais_de_autor_2019 = {}
for index, row in autores_pais_2019.iterrows():
    pais_de_autor_2019[row['Name']] = row['Country']

# Elimina duplicados en la información de autores
autores_2019 = autores_2019.drop_duplicates()

# Crea un grafo vacío utilizando NetworkX
G = nx.Graph()

# Inicializa diccionarios para contar la cantidad de autores por país y el número total de países
pais = {}
paises = 0

# Llena el diccionario 'pais' y cuenta la cantidad total de países
for index, row in autores_2019.iterrows():
    nombre = row['name1']
    paiss = pais_de_autor_2019[nombre]
    pais[paiss] = pais.get(paiss, 0) + 1
    paises += 1

# Inicializa variables para calcular el índice de homofilia
cross_edges = 0
total_edges = 0

# Calcula el número de aristas cruzadas y el número total de aristas en el grafo
for index, row in pares_autores_2019.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    if (pais_de_autor_2019[name1] != pais_de_autor_2019[name2]):
        cross_edges += 1
    total_edges += 1

# Calcula el índice de homofilia (e_ratio) y la probabilidad nula (mu)
e_ratio = cross_edges / total_edges
numerador = 0
for pp in pais:
    numerador += pais[pp] * (pais[pp] - 1)
mu = 1 - numerador / (paises * (paises - 1))

# Imprime el índice de homofilia y la probabilidad nula
print(e_ratio, mu)

# Compara e_ratio con mu y determina si hay homofilia o no
if e_ratio < mu:
    print("homofilia")
else:
    print("no homofilia")

0.569047619047619 0.9066928811337757
homofilia


In [23]:
pares_autores_2020 = colaboraciones_2020[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2020['pairs'] = pares_autores_2020['name1'] + ' - ' + pares_autores_2020['name2']
pares_autores_2020['pairs2'] = pares_autores_2020['name2'] + ' - ' + pares_autores_2020['name1']


# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2020.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2020.html')

graph_2020.html


In [24]:
# Extrae pares únicos de autores y sus afiliaciones
pares_autores_2020 = colaboraciones_2020[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2020['pairs'] = pares_autores_2020['name1'] + ' - ' + pares_autores_2020['name2']

# Extrae información única sobre autores y sus afiliaciones
autores_1_2020 = colaboraciones_2020[['name1', 'aff1']].drop_duplicates()
autores_2_2020 = colaboraciones_2020[['name2', 'aff2']].rename(columns={'name2': 'name1', 'aff2': 'aff1'}).drop_duplicates()
autores_2020 = pd.concat([autores_1_2020, autores_2_2020])

# Extraemos el nombre y el páis de cada autor
# Agregar Tabla de año 2020 con autores y países
autores_pais_2020 = authors_country_2020[['name', 'country']]

# Crea un diccionario que asigna el país a cada autor
pais_de_autor_2020 = {}
for index, row in autores_pais_2020.iterrows():
    pais_de_autor_2020[row['name']] = row['country']

# Elimina duplicados en la información de autores
autores_2020 = autores_2020.drop_duplicates()

# Crea un grafo vacío utilizando NetworkX
G = nx.Graph()

# Inicializa diccionarios para contar la cantidad de autores por país y el número total de países
pais = {}
paises = 0

# Llena el diccionario 'pais' y cuenta la cantidad total de países
for index, row in autores_2020.iterrows():
    nombre = row['name1']
    paiss = pais_de_autor_2020[nombre]
    pais[paiss] = pais.get(paiss, 0) + 1
    paises += 1

# Inicializa variables para calcular el índice de homofilia
cross_edges = 0
total_edges = 0

# Calcula el número de aristas cruzadas y el número total de aristas en el grafo
for index, row in pares_autores_2020.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    if (pais_de_autor_2020[name1] != pais_de_autor_2020[name2]):
        cross_edges += 1
    total_edges += 1

# Calcula el índice de homofilia (e_ratio) y la probabilidad nula (mu)
e_ratio = cross_edges / total_edges
numerador = 0
for pp in pais:
    numerador += pais[pp] * (pais[pp] - 1)
mu = 1 - numerador / (paises * (paises - 1))

# Imprime el índice de homofilia y la probabilidad nula
print(e_ratio, mu)

# Compara e_ratio con mu y determina si hay homofilia o no
if e_ratio < mu:
    print("homofilia")
else:
    print("no homofilia")

0.49236641221374045 0.9093002993417931
homofilia


In [25]:
pares_autores_2021 = colaboraciones_2021[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2021['pairs'] = pares_autores_2021['name1'] + ' - ' + pares_autores_2021['name2']
pares_autores_2021['pairs2'] = pares_autores_2021['name2'] + ' - ' + pares_autores_2021['name1']


# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2021.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2021.html')

graph_2021.html


In [26]:
pares_autores_2022 = colaboraciones_2022[['name1', 'aff1','name2', 'aff2', 'doi']].drop_duplicates()
pares_autores_2022['pairs'] = pares_autores_2022['name1'] + ' - ' + pares_autores_2022['name2']
pares_autores_2022['pairs2'] = pares_autores_2022['name2'] + ' - ' + pares_autores_2022['name1']

# Create an empty graph
G = nx.Graph()

# Add edges connecting name1 with name2
for index, row in pares_autores_2022.iterrows():
    name1 = row['name1']
    name2 = row['name2']
    G.add_edge(name1, name2)

# Create a pyvis network
nt = Network(notebook=True)

# Add nodes and edges to the network
for node in G.nodes:
    nt.add_node(node)

for edge in G.edges:
    nt.add_edge(edge[0], edge[1])

# Display the network
nt.show('graph_2022.html')

graph_2022.html
